In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas  as pd
import numpy as np
import matplotlib.pyplot  as plt
from sklearn.utils import shuffle
import cv2

import tensorflow as tf 
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense,Dropout, Flatten, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from keras import backend as k 
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping, ReduceLROnPlateau


In [ ]:
print(tf.__version__)

In [ ]:
data_path = '../input/cassava-leaf-disease-classification/'
train_csv_data_path = data_path + 'train.csv'
label_json_data_path = data_path + 'label_num_to_disease_map.json'
images_dir_data_path = data_path + 'train_images'

In [ ]:
train_csv = pd.read_csv(train_csv_data_path)
train_csv['label'] = train_csv['label'].astype('string')

label_class = pd.read_json(label_json_data_path, orient='index')
label_class = label_class.values.flatten().tolist()

In [ ]:
train_csv.head()

In [ ]:
print(label_class)

In [ ]:
print("Label names :")
for i, label in enumerate(label_class):
    print(f" {i}. {label}")

In [ ]:
# plot some train images
IMG_SIZE = 300
paths_show = [images_dir_data_path + "/" + train_csv.iloc[i]['image_id'] for i in range(6)]
print(paths_show)
plt.figure(figsize=(40,50))
for i in range(6):
    img = cv2.cvtColor(cv2.imread(paths_show[i]), cv2.COLOR_BGR2RGB)
    resized_img = cv2.resize(img, (IMG_SIZE, IMG_SIZE)).reshape(-1, IMG_SIZE, IMG_SIZE, 3)/255
    plt.subplot(4,2,i+1)
    plt.title('image_show')
    plt.imshow(resized_img[0])
    

## Data Augmentation and Preprocessing

In [ ]:
# Data augmentation and preprocessing
train_gen = ImageDataGenerator(
                                rotation_range=360,
                                width_shift_range=0.1,
                                height_shift_range=0.1,
                                brightness_range=[0.1,0.9],
                                shear_range=25,
                                zoom_range=0.3,
                                channel_shift_range=0.1,
                                horizontal_flip=True,
                                vertical_flip=True,
                                rescale=1/255,
                                validation_split=0.15

                              )

valid_gen = ImageDataGenerator(rescale=1/255,
                               validation_split = 0.15
                              )

In [ ]:
BATCH_SIZE = 32
IMG_SIZE = 512


In [ ]:
train_generator = train_gen.flow_from_dataframe(
                            dataframe=train_csv,
                            directory = images_dir_data_path,
                            x_col = "image_id",
                            y_col = "label",
                            target_size = (IMG_SIZE, IMG_SIZE),
                            class_mode = "categorical",
                            batch_size = BATCH_SIZE,
                            shuffle = True,
                            subset = "training",

)

valid_generator = valid_gen.flow_from_dataframe(
                            dataframe=train_csv,
                            directory = images_dir_data_path,
                            x_col = "image_id",
                            y_col = "label",
                            target_size = (IMG_SIZE, IMG_SIZE),
                            class_mode = "categorical",
                            batch_size = BATCH_SIZE,
                            shuffle = False,
                            subset = "validation"
)

### Visualizing Train Set

In [ ]:
batch = next(train_generator)
images = batch[0]
labels = batch[1]

In [ ]:
print(len(train_generator))
print(len(valid_generator))

In [ ]:
plt.figure(figsize=(12, 9))
# only show 6 images
for i, (img, label) in enumerate(zip(images, labels)):
    # label is like : [0. 0. 0. 1. 0.]
    plt.subplot(2,3, i%6+1)
    plt.axis('off')
    plt.imshow(img)
    plt.title(label_class[np.argmax(label)])
    if i == 7:
        break
    
    
    

In [ ]:
# for Xception
IMG_SIZE = 512
model_xception = tf.keras.applications.Xception(include_top=False, input_shape=(IMG_SIZE,IMG_SIZE,3), weights=None)

model_xception.save_weights('xceptionweights.h5')

model_xception.load_weights('xceptionweights.h5')
model_xception.summary()

## Fine Tune Last layer of Xception

In [ ]:
for layer in model_xception.layers:
    layer.trainable=False
model_xception.layers[-2].trainable = True
model_xception.layers[-3].trainable = True


#Adding custom Layers 
x = model_xception.output
x = GlobalAveragePooling2D()(x)
# x = Dense(256, activation='relu')(x)
# x = Dropout(0.3)(x)
x = Dense(256, activation="elu")(x)
x = Dropout(0.3)(x)
x = Dense(50, activation="elu")(x)
x = Dropout(0.1)(x)
predictions = Dense(5, activation="softmax")(x)

# creating the final model 
model_final = Model(inputs = model_xception.input, outputs = predictions)
model_final.summary()

In [ ]:
from tensorflow.keras.optimizers import RMSprop
# compile the model 
model_final.compile(loss = "categorical_crossentropy", optimizer =RMSprop(lr=0.001), metrics=["accuracy"])

In [ ]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=2, min_lr=0.000001)
mcp_save = ModelCheckpoint('model_train.hdf5', save_best_only=True, monitor='acc', mode='max')

In [ ]:
STEPS_PER_EPOCH = len(train_generator) // BATCH_SIZE
VALID_STEPS = len(valid_generator) // BATCH_SIZE


## Training

In [ ]:
history = model_final.fit(
      train_generator,
      steps_per_epoch=STEPS_PER_EPOCH,
      epochs=20,
      validation_data=valid_generator,
      batch_size= VALID_STEPS,
      callbacks=[mcp_save]
      )

model_final.save('model_xception_fine_last_layerv1.h5')

### Visualization Accuracy and Loss


In [ ]:
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

## Predictions

In [ ]:

test_img_path = data_path+"test_images/2216849948.jpg"
img = cv2.imread(test_img_path)
resized_img = cv2.resize(img, (IMG_SIZE, IMG_SIZE)).reshape(-1, IMG_SIZE, IMG_SIZE, 3)/255.
plt.figure(figsize=(8,4))
plt.title("TEST IMAGE")
plt.imshow(resized_img[0])

In [ ]:
preds = []
ss = pd.read_csv(data_path+'sample_submission.csv')

for image in ss.image_id:
    img = tf.keras.preprocessing.image.load_img(data_path+'test_images/' + image)
    img = tf.keras.preprocessing.image.img_to_array(img)
    img = tf.keras.preprocessing.image.smart_resize(img, (IMG_SIZE, IMG_SIZE))
    img = tf.reshape(img, (-1, IMG_SIZE, IMG_SIZE, 3))
    prediction = model_final.predict(img/255)
    preds.append(np.argmax(prediction))

my_submission = pd.DataFrame({'image_id': ss.image_id, 'label': preds})
my_submission.to_csv('submission.csv', index=False) 

In [ ]:
#  for EfficientNetB7
# model_effiB7 = tf.keras.applications.EfficientNetB7(include_top=False, input_shape=(IMG_SIZE,IMG_SIZE, 3))



In [ ]:
# for i, layer in enumerate(model_effiB7.layers):
#     print("{}:  {}".format(i, layer))

### Fine tune last layer of efficientnet B7

In [ ]:
# for layer in model_effiB7.layers:
#     layer.trainable=False
# model_effiB7.layers[-2].trainable = True
# model_effiB7.layers[-3].trainable = True


# #Adding custom Layers 
# x = model_effiB7.output
# x = GlobalAveragePooling2D()(x)
# # x = Dense(256, activation='relu')(x)
# # x = Dropout(0.3)(x)
# x = Dense(256, activation="elu")(x)
# x = Dropout(0.3)(x)
# x = Dense(50, activation="elu")(x)
# x = Dropout(0.1)(x)
# predictions = Dense(5, activation="softmax")(x)

# # creating the final model 
# model_final = Model(inputs = model_effiB7.input, outputs = predictions)
# model_final.summary()

In [ ]:
# from tensorflow.keras.optimizers import RMSprop
# # compile the model 
# model_final.compile(loss = "categorical_crossentropy", optimizer =RMSprop(lr=0.001), metrics=["accuracy"])

We will penalize our learning rate,if the validation accuracy does not improve after 2 epoch by 10% and finally train it for given epochs with batch_size=BATCH_SIZE

In [ ]:
# reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
#                               patience=2, min_lr=0.000001)
# mcp_save = ModelCheckpoint('model_train.hdf5', save_best_only=True, monitor='acc', mode='max')

### Training

In [ ]:
# history = model_final.fit(
#       train_generator,
#       steps_per_epoch=BATCH_SIZE + 10,#len(train_generator)//BATCH_SIZE,
#       epochs=20,
#       validation_data=valid_generator,
#       batch_size= BATCH_SIZE,#len(valid_generator)//BATCH_SIZE,
#       callbacks=[reduce_lr, mcp_save]
#       )

# model_final.save('model_efficentNetB7fine_last_layerv1.h5')

### Visulaization Accuracy and Loss

In [ ]:
# print(history.history.keys())
# # summarize history for accuracy
# plt.plot(history.history['accuracy'])
# plt.plot(history.history['val_accuracy'])
# plt.title('model accuracy')
# plt.ylabel('accuracy')
# plt.xlabel('epoch')
# plt.legend(['train', 'test'], loc='upper left')
# plt.show()
# # summarize history for loss
# plt.plot(history.history['loss'])
# plt.plot(history.history['val_loss'])
# plt.title('model loss')
# plt.ylabel('loss')
# plt.xlabel('epoch')
# plt.legend(['train', 'test'], loc='upper left')
# plt.show()

### Predictions

In [ ]:
# test_img_path = data_path+"test_images/2216849948.jpg"

# img = cv2.imread(test_img_path)
# resized_img = cv2.resize(img, (IMG_SIZE, IMG_SIZE)).reshape(-1, IMG_SIZE, IMG_SIZE, 3)/255

# plt.figure(figsize=(8,4))
# plt.title("TEST IMAGE")
# plt.imshow(resized_img[0])

In [ ]:
# preds = []
# ss = pd.read_csv(data_path+'sample_submission.csv')

# for image in ss.image_id:
#     img = tf.keras.preprocessing.image.load_img(data_path+'test_images/' + image)
#     img = tf.keras.preprocessing.image.img_to_array(img)
#     img = tf.keras.preprocessing.image.smart_resize(img, (IMG_SIZE, IMG_SIZE))
#     img = tf.reshape(img, (-1, IMG_SIZE, IMG_SIZE, 3))
#     prediction = model_final.predict(img/255)
#     preds.append(np.argmax(prediction))

# my_submission = pd.DataFrame({'image_id': ss.image_id, 'label': preds})
# my_submission.to_csv('submission.csv', index=False) 